In [478]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder 
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

survey= pd.read_csv("data_nonpersonal.csv", names=["time","age","country","city","sex","commute_type","commute_time","sit_ratio","sleep_sat","sleep_pos","activity","tracker","posture","posture_concern","pain_freq","pain_severe","pain_loc","pain_time","medical_type","like_least"],header=0)
survey

,time,age,country,city,sex,commute_type,commute_time,sit_ratio,sleep_sat,sleep_pos,activity,tracker,posture,posture_concern,pain_freq,pain_severe,pain_loc,pain_time,medical_type,like_least
0,2019/10/14 12:01:24 PM EST,22,United States,Lynchburg,Male,Drive a car,10 - 29 minutes,< 25%,Neutral,On your side,Active (some form of exercise 3 - 4 days per w...,No,2.0,"Probably, but I likely wouldn't do anything ab...",3,2.0,Lower back,While standing;While moving;While sedentary,"Taken ""over the counter"" pain medication",Treatment is inconvenient and time consuming
1,2019/10/14 9:28:10 PM EST,24,United States,Saint Louis,Male,Walk,10 - 29 minutes,75% - 100%,Satisfied,On your stomach,Athletic (daily exercise),No,3.0,"Probably, but I likely wouldn't do anything ab...",4,3.0,Upper back / shoulders,While sedentary;While sleeping,"Taken ""over the counter"" pain medication;Used ...",Lack of access to cost effective treatment
2,2019/10/15 5:51:26 AM EST,20,Spain,Madrid,Female,Walk;Use public transportation,30 - 45 minutes,50% - 75%,Satisfied,On your side,Moderate (some form of exercise 1 - 2 days per...,No,2.0,No,2,2.0,Upper back / shoulders,While moving;While sedentary,Not Applicable,I havenÃÂ´t sought back pain relief
3,2019/10/15 9:24:55 AM EST,26,USA,Toledo,Female,Drive a car,10 - 29 minutes,50% - 75%,Satisfied,On your side,Moderate (some form of exercise 1 - 2 days per...,No,3.0,No,2,2.0,Upper back / shoulders,While standing;While sedentary,Not Applicable,Unsure what to do or where to begin
4,2019/10/15 9:33:24 AM EST,57,USA,Palos Verdes Estates,Male,Drive a car,10 - 29 minutes,25% - 50%,Neutral,On your side,Moderate (some form of exercise 1 - 2 days per...,Yes,4.0,No,3,2.0,Lower back,While standing,Used homeopathic remedies,Doing frequent exercises / stretches
5,2019/10/15 9:35:49 AM EST,29,USA,Fort Myers,Female,Drive a car,30 - 45 minutes,50% - 75%,Dissatisfied,On your stomach,Moderate (some form of exercise 1 - 2 days per...,Yes,4.0,No,5,5.0,Lower back,While moving;While sedentary,Visited a physician or doctor;Visited a physic...,Treatment is inconvenient and time consuming
6,2019/10/15 9:44:38 AM EST,30,United States,NaN,Female,Drive a car,> 45 minutes,75% - 100%,Satisfied,On your back,Minimal (walking to and from the car/bus each ...,Yes,3.0,Yes,4,3.0,Upper back / shoulders;Base of the neck;Lower ...,While moving;While sleeping,Visited a physician or doctor;Visited a physic...,Lack of professional knowledge
7,2019/10/15 9:59:20 AM EST,56,usa,NaN,Male,Drive a car,< 10 minutes,75% - 100%,Neutral,On your side,Minimal (walking to and from the car/bus each ...,No,3.0,No,3,3.0,Lower back,While standing,Visited a physician or doctor,Lack of access to cost effective treatment
8,2019/10/15 10:22:03 AM EST,21,United States,Southfield,Female,Walk;Bike,< 10 minutes,75% - 100%,Dissatisfied,On your side,Minimal (walking to and from the car/bus each ...,No,3.0,Yes,2,2.0,Upper back / shoulders;Lower back,While standing;While sedentary,"Taken ""over the counter"" pain medication",Treatment is inconvenient and time consuming
9,2019/10/15 10:36:06 AM EST,48,United States,Fort Myers,Female,Drive a car,10 - 29 minutes,75% - 100%,Dissatisfied,On your back,Moderate (some form of exercise 1 - 2 days per...,No,2.0,Yes,3,2.0,Upper back / shoulders;Base of the neck,While standing;While sedentary,Visited a physician or doctor;Visited a physic...,There is virtually no treatment that corrects ...


## Cleaning columns

In [479]:
survey["country"]=survey.country.str.lower()
country_us=survey.country.str.match(".*usa.*|.*united states.*|us|u\.s\.|u\.s|.*united states of america.*|.*u\.s\.a.*|.*america.*")
for i in range(0,len(country_us)):
    if country_us[i]:
        survey.loc[i,"country"]="usa"
        


In [468]:
surveysub= survey[["country","sex","commute_type","commute_time","sit_ratio","sleep_sat","sleep_pos","activity","tracker","posture","posture_concern","pain_freq","pain_severe","pain_loc","pain_time","medical_type"]]

### Creating Categorical Manual

In [469]:
#commutes
surveysub["commute_bike"]=[0] * len(surveysub["commute_type"])
surveysub["commute_walk"]=[0] * len(surveysub["commute_type"])
surveysub["commute_drive"]=[0] * len(surveysub["commute_type"])
surveysub["commute_public"]=[0] * len(surveysub["commute_type"])
#pain locations
surveysub["pain_neck"]=[0] * len(surveysub["pain_loc"])
surveysub["pain_lower"]=[0] * len(surveysub["pain_loc"])
surveysub["pain_shoulder"]=[0] * len(surveysub["pain_loc"])
#pain experiences
surveysub["pain_stand"]=[0] * len(surveysub["pain_time"])
surveysub["pain_move"]=[0] * len(surveysub["pain_time"])
surveysub["pain_sleep"]=[0] * len(surveysub["pain_time"])
surveysub["pain_sed"]=[0] * len(surveysub["pain_time"])
#medication types
surveysub["med_doc"]=[0] * len(surveysub["medical_type"])
surveysub["med_pt"]=[0] * len(surveysub["medical_type"])
surveysub["med_cbd"]=[0] * len(surveysub["medical_type"])
surveysub["med_chiro"]=[0] * len(surveysub["medical_type"])
surveysub["med_presc"]=[0] * len(surveysub["medical_type"])
surveysub["med_counter"]=[0] * len(surveysub["medical_type"])
surveysub["med_homeo"]=[0] * len(surveysub["medical_type"])

C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [470]:
for c in range(0,len(surveysub["commute_type"])):
    typec=surveysub["commute_type"][c]
    painl=surveysub["pain_loc"][c]
    paint=surveysub["pain_time"][c]
    med=surveysub["medical_type"][c]
    sleep=surveysub["sleep_pos"][c]
    try:
        typesc=typec.split(";")
        #print(typesc)
    except:
        typesc=[0]
    try:
        painsl=painl.split(";")
        #print(typesc)
    except:
        painsl=[0]

    try:
        paints=paint.split(";")
        #print(typesc)
    except:
        painst=[0]
    try:
        meds=med.split(";")
        #print(typesc)
    except:
        meds=[0]
   
        
    for com in typesc:
        if com =="Walk":
            surveysub["commute_walk"][c]=1
        elif com =="Bike":
            surveysub["commute_bike"][c]=1
        elif com =="Drive a car":
            surveysub["commute_drive"][c]=1
        elif com =="Use public transportation":
            surveysub["commute_public"][c]=1
    for pa in painsl:
        if pa =="Upper back / shoulders":
            surveysub["pain_shoulder"][c]=1
        elif pa =="Lower back":
            surveysub["pain_lower"][c]=1
        elif pa =="Base of the neck":
            surveysub["pain_neck"][c]=1
    for pt in paints:
        if pt =="While standing":
            surveysub["pain_stand"][c]=1
        elif pt =="While moving":
            surveysub["pain_move"][c]=1
        elif pt =="While sedentary":
            surveysub["pain_sed"][c]=1
        elif pt =="While sleeping":
            surveysub["pain_sleep"][c]=1
    for m in meds:
        if m =="Visited a physician or doctor":
            surveysub["med_doc"][c]=1
        elif m =="Visited a physical therapist":
            surveysub["med_pt"][c]=1
        elif m =="Visited a chiropractor":
            surveysub["med_chiro"][c]=1
        elif m =="Taken prescription pain medication":
            surveysub["med_presc"][c]=1
        elif m =="Taken \"over the counter\" pain medication":
            surveysub["med_counter"][c]=1
        elif m =="Used homeopathic remedies":
            surveysub["med_homeo"][c]=1
        elif m =="Used CBD products":
            surveysub["med_cbd"][c]=1



C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

### Create Ordinal
commute time
sitting ratio
physical activity
sleep sat


In [471]:
for i in range(0, len(surveysub["commute_time"])):
    com=surveysub["commute_time"][i]
    sit=surveysub["sit_ratio"][i]
    physical= surveysub["activity"][i]
    sleep=surveysub["sleep_sat"][i]

    
    if com =="< 10 minutes":
        surveysub["commute_time"][i]=0
    elif com=="10 - 29 minutes":
        surveysub["commute_time"][i]=1
    elif com=="30 - 45 minutes": 
        surveysub["commute_time"][i]=2
    elif com=="> 45 minutes": 
        surveysub["commute_time"][i]=3
    else:
        surveysub["commute_time"][i]=None
        
    if sit=="< 25%":
        surveysub["sit_ratio"][i]=0
    elif sit=="25% - 50%":
        surveysub["sit_ratio"][i]=1
    elif sit=="50% - 75%":
        surveysub["sit_ratio"][i]=2
    elif sit=="75% - 100%":
        surveysub["sit_ratio"][i]=3
    else:
        surveysub["sit_ratio"][i]=None        
    
    if physical == "Minimal (walking to and from the car/bus each day)":
        surveysub["activity"][i]=0
    elif physical == "Moderate (some form of exercise 1 - 2 days per week)":
        surveysub["activity"][i]=1
    elif physical == "Active (some form of exercise 3 - 4 days per week)":
        surveysub["activity"][i]=2
    elif physical =="Very active (some form of exercise 5 days a week)" or physical =="Very active (some from of exercise 5 days a week)":
        surveysub["activity"][i]=3
    elif physical =="Athletic (daily exercise)":
        surveysub["activity"][i]=4
    else:
        surveysub["activity"][i]=None

    if sleep =="Very dissatisfied":
        surveysub["sleep_sat"][i]=0
    elif sleep=="Dissatisfied":
        surveysub["sleep_sat"][i]=1
    elif sleep=="Neutral": 
        surveysub["sleep_sat"][i]=2
    elif sleep=="Satisfied": 
        surveysub["sleep_sat"][i]=3
    elif sleep=="Very satisfied": 
        surveysub["sleep_sat"][i]=4
    else:
        surveysub["sleep_sat"][i]=None
    
    

C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A v

In [472]:
surveydummies=pd.get_dummies(surveysub[["sex","sleep_pos","posture_concern","country", "tracker"]]) 
surveysubd= surveysub[["commute_time","sit_ratio","sleep_sat",
                       "activity","posture","pain_freq","pain_severe",
                       "commute_bike","commute_walk","commute_drive","commute_public",
                       "pain_neck","pain_lower","pain_shoulder","pain_stand","pain_move",
                       "pain_sleep","pain_sed","med_doc","med_pt","med_cbd","med_chiro",
                       "med_presc","med_counter","med_homeo"]]
surveysubd=pd.concat([surveysubd,surveydummies,survey["age"]], axis=1)


## Write to CSV

In [473]:
surveysubd.to_csv("cleaned.csv", index=False)

In [481]:
survey.to_csv("grouped.csv", index=False)